# FEATURES

In [65]:
from datasets import get_dataset_split_names
from datasets import load_dataset
from datasets import dataset_dict
from datasets import arrow_dataset
import string

dataset = load_dataset("imdb")

Found cached dataset imdb (/home/amine/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 956.15it/s]


In [66]:
document = dataset["train"][0]
document

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

1- Preprocess function

In [67]:
def preprocess(data : dict) -> dict:
    '''Takes a document from a dataset, lowers the letters and
    replace all punctuations by spaces'''
    text = data["text"]
    for character in string.punctuation:
        if character != "-" and character != "!":
          text = text.replace(character, ' ')
    data["text"] = text.lower()
    return data

In [68]:
updated_dataset= dataset.map(preprocess)

Loading cached processed dataset at /home/amine/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-a1ff6ac6f3dae5b2.arrow
Loading cached processed dataset at /home/amine/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-3ff24866f561b56d.arrow
Loading cached processed dataset at /home/amine/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-c8e6bf051898815c.arrow


2- Naive Bayes classifier from scratch

In [69]:
import math
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def build_vocabulary_string(Vocabulary : dict[int], text : string, text_category : int, Categories : set[int]):
    '''Updates the Vocabulary by adding the words contained in the text'''
    word_count = 0
    features_vector = [0,0,0,0,0,0,0]
    
    Analyser = SentimentIntensityAnalyzer()
    #test_dict = Analyser.polarity_scores(text)
    
    for word in text.split():
        if not (word in Vocabulary) :
            Vocabulary[word] = {}
            for category in Categories:
                Vocabulary[word][category] = 0
        word_count += 1
        if (word == "no") :
            features_vector[0] = 1
        if (word == "i" or word == "you") :
            features_vector[1] +=  1
        if (word == "no") :
            features_vector[2] = 1
        
        test_dict = Analyser.polarity_scores(word)
        if (test_dict["compound"] >= 0.05):
            features_vector[4] += 1
        elif (test_dict["compound"] <= -0.05):
            features_vector[5] += 1
        else : #BONUS FEATURE, number of neutral lexicon
            features_vector[6] += 1
    
        
        Vocabulary[word][text_category] += 1
    features_vector[3] = math.log(word_count)
    return Vocabulary, features_vector

def build_vocabulary(dataset : dataset_dict.DatasetDict, Categories : set[int]) -> dict[int]:
    '''Construct a vocabulary from the documents contained in dataset'''
    Vocabulary = {}
    for document in dataset:
        Vocabulary, features_vector= build_vocabulary_string(Vocabulary, document["text"], document["label"], Categories)
    return Vocabulary, features_vector

In [70]:
categories = {0,1}
voc, feat = build_vocabulary(updated_dataset["train"],categories)


# II REGRESSION LINEAIRE